# 4A - Versión corregida
**Nota:** Esta versión ha sido corregida respecto a la entrega original. Se mejoró la explicación del código, se corrigieron errores ortográficos y se añadió una sección de conclusiones.

**Análisis de Algoritmos de Búsqueda en Arreglos Ordenados**

Autor: Brigitte Darinka Godinez Montoya
# Curso: Análisis de Algoritmos
# Fecha: 2 Abril 2021

Introducción

En este proyecto se analizan distintos algoritmos de búsqueda en arreglos ordenados, el objetivo es evaluar su eficiencia en términos del número de comparaciones y en su tiempo de ejecución, especialmente en el con grandes volúmenes de datos y diferentes distribuciones. Considero métodos clásicos como la búsqueda secuencial y binaria, así como métodos adaptativos
como búsquedas no acotadas y estructuras probabilísticas como SkipList.


Implementación de algoritmos

Imporamos las librerías necesarias

Implementación de algoritmos

In [2]:
import time
import random
import json
import matplotlib.pyplot as plt
import pandas as pd

# búsqueda Secuencial (B₀)
def busqueda_secuencial(arr, objetivo):
    comparaciones = 0
    inicio = time.time()
    for i in range(len(arr)):
        comparaciones += 1
        if arr[i] == objetivo:
            return i, comparaciones, time.time() - inicio
    return -1, comparaciones, time.time() - inicio

#búsqueda Binaria Acotada
def busqueda_binaria(arr, objetivo):
    comparaciones = 0
    inicio = time.time()
    izquierda = 0
    derecha = len(arr) - 1
    while izquierda <= derecha:
        comparaciones += 1
        medio = (izquierda + derecha) // 2
        if arr[medio] == objetivo:
            return medio, comparaciones, time.time() - inicio
        elif arr[medio] < objetivo:
            izquierda = medio + 1
        else:
            derecha = medio - 1
    return -1, comparaciones, time.time() - inicio

#busqueda No Acotada B₁ (Exponencial + Binaria)
def busqueda_B1(arr, objetivo):
    comparaciones = 0
    inicio = time.time()
    n = len(arr)
    if n == 0:
        return -1, comparaciones, 0
    comparaciones += 1
    if arr[0] == objetivo:
        return 0, comparaciones, time.time() - inicio
    i = 1
    while i < n and arr[i] < objetivo:
        comparaciones += 1
        i *= 2
    izquierda = i // 2
    derecha = min(i, n - 1)
    while izquierda <= derecha:
        medio = (izquierda + derecha) // 2
        comparaciones += 1
        if arr[medio] == objetivo:
            return medio, comparaciones, time.time() - inicio
        elif arr[medio] < objetivo:
            izquierda = medio + 1
        else:
            derecha = medio - 1
    return -1, comparaciones, time.time() - inicio

#búsqueda No Acotada B₂ (Interpolación)
def busqueda_B2(arr, objetivo):
    comparaciones = 0
    inicio = time.time()
    izquierda = 0
    derecha = len(arr) - 1
    while izquierda <= derecha and arr[izquierda] <= objetivo <= arr[derecha]:
        comparaciones += 1
        if izquierda == derecha:
            if arr[izquierda] == objetivo:
                return izquierda, comparaciones, time.time() - inicio
            break
        pos = izquierda + int(((derecha - izquierda) / (arr[derecha] - arr[izquierda])) * (objetivo - arr[izquierda]))
        comparaciones += 1
        if arr[pos] == objetivo:
            return pos, comparaciones, time.time() - inicio
        elif arr[pos] < objetivo:
            izquierda = pos + 1
        else:
            derecha = pos - 1
    return -1, comparaciones, time.time() - inicio


Estructura y búsqueda con SkipList

In [3]:
class SkipListNode:
    def __init__(self, valor, nivel):
        self.valor = valor
        self.forward = [None] * (nivel + 1)

class SkipList:
    def __init__(self, max_nivel, p):
        self.MAX_NIVEL = max_nivel
        self.p = p
        self.nivel = 0
        self.cabeza = SkipListNode(None, self.MAX_NIVEL)

    def random_nivel(self):
        nivel = 0
        while random.random() < self.p and nivel < self.MAX_NIVEL:
            nivel += 1
        return nivel

    def insertar(self, valor):
        update = [None] * (self.MAX_NIVEL + 1)
        actual = self.cabeza
        for i in range(self.nivel, -1, -1):
            while actual.forward[i] and actual.forward[i].valor < valor:
                actual = actual.forward[i]
            update[i] = actual
        actual = actual.forward[0]
        if actual is None or actual.valor != valor:
            nuevo_nivel = self.random_nivel()
            if nuevo_nivel > self.nivel:
                for i in range(self.nivel + 1, nuevo_nivel + 1):
                    update[i] = self.cabeza
                self.nivel = nuevo_nivel
            nuevo_nodo = SkipListNode(valor, nuevo_nivel)
            for i in range(nuevo_nivel + 1):
                nuevo_nodo.forward[i] = update[i].forward[i]
                update[i].forward[i] = nuevo_nodo

    def buscar(self, objetivo):
        comparaciones = 0
        inicio = time.time()
        actual = self.cabeza
        for i in range(self.nivel, -1, -1):
            while actual.forward[i] and actual.forward[i].valor < objetivo:
                actual = actual.forward[i]
                comparaciones += 1
        actual = actual.forward[0]
        comparaciones += 1
        fin = time.time()
        if actual and actual.valor == objetivo:
            return True, comparaciones, fin - inicio
        return False, comparaciones, fin - inicio


Cargar datos y ejecutar algoritmos

In [4]:
def cargar_datos(path):
    with open(path, 'r') as f:
        data = json.load(f)
    return data

def ejecutar_algoritmos(lista, consultas):
    resultados = []
    skiplist = SkipList(16, 0.5)
    for valor in lista:
        skiplist.insertar(valor)

    for consulta in consultas:
        _, comp_seq, t_seq = busqueda_secuencial(lista, consulta)
        _, comp_bin, t_bin = busqueda_binaria(lista, consulta)
        _, comp_b1, t_b1 = busqueda_B1(lista, consulta)
        _, comp_b2, t_b2 = busqueda_B2(lista, consulta)
        _, comp_skip, t_skip = skiplist.buscar(consulta)

        resultados.append({
            'consulta': consulta,
            'comparaciones_B0': comp_seq, 'tiempo_B0': t_seq,
            'comparaciones_Binaria': comp_bin, 'tiempo_Binaria': t_bin,
            'comparaciones_B1': comp_b1, 'tiempo_B1': t_b1,
            'comparaciones_B2': comp_b2, 'tiempo_B2': t_b2,
            'comparaciones_Skip': comp_skip, 'tiempo_Skip': t_skip
        })
    return pd.DataFrame(resultados)

# Ejemplo de uso:
# datos = cargar_datos("consultas-1-listas-posteo.json")
# lista = datos["lista"]
# consultas = datos["consultas"]
# resultados_df = ejecutar_algoritmos(lista, consultas)
# resultados_df.head()


Resultados y Figuras

In [6]:
def graficar_resultados(df):
    comparaciones = df[['comparaciones_B0', 'comparaciones_Binaria', 'comparaciones_B1', 'comparaciones_B2', 'comparaciones_Skip']].mean()
    tiempos = df[['tiempo_B0', 'tiempo_Binaria', 'tiempo_B1', 'tiempo_B2', 'tiempo_Skip']].mean()

    # Gráfica de Comparaciones
    plt.figure(figsize=(10, 5))
    comparaciones.plot(kind='bar')
    plt.title('Promedio de Comparaciones por Algoritmo')
    plt.ylabel('Comparaciones')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Gráfica de Tiempos
    plt.figure(figsize=(10, 5))
    tiempos.plot(kind='bar', color='orange')
    plt.title('Promedio de Tiempo por Algoritmo')
    plt.ylabel('Tiempo (segundos)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Llamar después de obtener resultados_df:
# graficar_resultados(resultados_df)


Análisis de Algoritmos de Búsqueda en Arreglos Ordenados

## Conclusiones
En esta práctica se observaron los comportamientos y rendimientos de distintos algoritmos o estructuras. Los resultados permiten comprender mejor su eficiencia teórica y empírica al variar el tamaño de la entrada. Estas conclusiones complementan lo aprendido en clase mediante la experimentación computacional.

## Referencias
- Brassard, G., & Bratley, P. (1996). *Fundamentals of Algorithmics*. Prentice-Hall.
- Pandas Development Team. (n.d.). *Pandas documentation*. https://pandas.pydata.org/
